In [2]:
import pennylane as qml
import numpy as np # Using pennylane's wrapped numpy
from autograd.numpy.numpy_boxes import ArrayBox
import matplotlib.pyplot as plt
import os
from functools import partial
from qiskit.circuit.library import *
from qiskit import *
from qiskit.quantum_info import *
import autograd
from pennylane.wires import Wires
import matplotlib.cm as cm
import base64
from qiskit import *
from qiskit.quantum_info import *
import pickle
from qutip import *
from qutip.qip.operations import cnot,rz,rx,ry,snot,toffoli,fredkin
from qutip.qip.circuit import QubitCircuit
from scipy.optimize import minimize
from sympy import symbols, MatrixSymbol, lambdify, Matrix, pprint
import random
import pickle
import base64
import scipy
import time
from numpy.random import randn
from pathos.multiprocessing import Pool, ProcessingPool, ThreadingPool
from pathos.pools import ProcessPool
from multiprocessing import cpu_count
from scipy.stats import unitary_group
import pandas as pd
from pathlib import Path
from matplotlib.ticker import ScalarFormatter
from qutip import entropy_vn, concurrence

from sympy import Matrix,pprint
from autograd.numpy.numpy_boxes import ArrayBox
import matplotlib.pyplot as plt
import os
from functools import partial
from qiskit.circuit.library import *
from qiskit import *
from qiskit.quantum_info import *
import autograd
from pennylane.wires import Wires
import matplotlib.cm as cm
import base64
from multiprocessing import Pool, cpu_count
from qiskit import *
from qiskit.quantum_info import *
import pickle
from qutip import *
from qutip.qip.operations import cnot,rz,rx,ry,snot,toffoli,fredkin
from qutip.qip.circuit import QubitCircuit
from scipy.optimize import minimize
from sympy import symbols, MatrixSymbol, lambdify, Matrix, pprint
import random
import dill as pickle
import base64
import time
from numpy.random import randn
from pathos.multiprocessing import Pool, ProcessingPool, ThreadingPool
from pathos.pools import ProcessPool
from multiprocessing import cpu_count
from scipy.stats import unitary_group
import pandas as pd
from pathlib import Path
from matplotlib.ticker import ScalarFormatter
from qutip import entropy_vn, concurrence
import seaborn as sns

In [35]:
import pickle
from pathlib import Path
import numpy as np
import statistics
import jax.numpy as jnp
def convert_to_float(value):
    """Convert NumPy arrays or other numeric types to float or list of floats."""
    if isinstance(value, np.ndarray):
        return [float(f"{x:.13f}") for x in value]
    elif isinstance(value, (float, int, np.float32, np.float64, np.int32, np.int64)):
        return float(f"{value:.13f}")
    else:
        return value
def compute_qfim_entropy(eigvals, threshold=1e-12):
    # Filter eigenvalues that are larger than the threshold
    filtered_eigvals = eigvals[eigvals > threshold]
    
    # Normalize the eigenvalues to form a probability distribution
    normalized_eigvals = filtered_eigvals / jnp.sum(filtered_eigvals)
    
    # Compute the entropy
    entropy = -jnp.sum(normalized_eigvals * jnp.log(normalized_eigvals + 1e-12))  # Adding a small value to avoid log(0)
    
    return entropy
def get_qfim_eigvals(file_path):
    file_path = Path(file_path) if not isinstance(file_path, Path) else file_path
    if not file_path.exists():
        print(f"File {file_path} does not exist.")
        return None

    with open(file_path, 'rb') as f:
        df = pickle.load(f)

    qfim_eigvals_dict = {}
    fixed_param_values = {}
    for fixed_params_dict in df.keys():
        #if fixed_params_dict == 'fixed_params2':
        #    continue
        
        for trainable_params_dict in df[fixed_params_dict].keys():
          #  if trainable_params_dict == 'test14' or trainable_params_dict == 'test48':
         #       continue
            results = df[fixed_params_dict][trainable_params_dict]
            fixed_param_values[fixed_params_dict] = results['fixed_params']
            if 'qfim_eigvals' in results:
                qfim_eigvals = results['qfim_eigvals']
                if fixed_params_dict not in qfim_eigvals_dict:
                    qfim_eigvals_dict[fixed_params_dict] = {}
                qfim_eigvals_dict[fixed_params_dict][trainable_params_dict] = qfim_eigvals

    return qfim_eigvals_dict, fixed_param_values

def get_jacobian(file_path):
    file_path = Path(file_path) if not isinstance(file_path, Path) else file_path
    if not file_path.exists():
        print(f"File {file_path} does not exist.")
        return None

    with open(file_path, 'rb') as f:
        df = pickle.load(f)

    jacobian_dict = {}
    for fixed_params_dict in df.keys():
        #if fixed_params_dict == 'fixed_params2':
        #    continue
        for trainable_params_dict in df[fixed_params_dict].keys():
            #if trainable_params_dict == 'test14' or trainable_params_dict == 'test48':
             #   continue
            results = df[fixed_params_dict][trainable_params_dict]
            if 'jacobian' in results:
                jacobian = results['jacobian']
                if fixed_params_dict not in jacobian_dict:
                    jacobian_dict[fixed_params_dict] = {}
                jacobian_dict[fixed_params_dict][trainable_params_dict] = jacobian

    return jacobian_dict
def find_jacobian_variance_extremes(jacobian_dict):
    min_var_val = float('inf')
    max_var_val = float('-inf')

    min_var_params = None
    max_var_params = None

    for fixed_param, trainable_dict in jacobian_dict.items():
        for trainable_param, jacobian in trainable_dict.items():
            variance = np.var(jacobian)

            if variance < min_var_val:
                min_var_val = variance
                min_var_params = (fixed_param, trainable_param)

            if variance > max_var_val:
                max_var_val = variance
                max_var_params = (fixed_param, trainable_param)

    return {
        "min_jacobian_variance": (min_var_params, min_var_val),
        "max_jacobian_variance": (max_var_params, max_var_val)
    }
def find_extreme_params(qfim_eigvals_dict):
    min_trace_val = float('inf')
    max_trace_val = float('-inf')
    min_var_val = float('inf')
    max_var_val = float('-inf')

    min_trace_params = None
    max_trace_params = None
    min_var_params = None
    max_var_params = None
    threshold =1e-12
    for fixed_param, trainable_dict in qfim_eigvals_dict.items():
        for trainable_param, eigvals in trainable_dict.items():
            trace = np.sum(eigvals)
            variance = np.var(eigvals[eigvals > threshold])
            

            if trace < min_trace_val:
                min_trace_val = trace
                min_trace_params = (fixed_param, trainable_param)

            if trace > max_trace_val:
                max_trace_val = trace
                max_trace_params = (fixed_param, trainable_param)

            if variance < min_var_val:
                min_var_val = variance
                min_var_params = (fixed_param, trainable_param)

            if variance > max_var_val:
                max_var_val = variance
                max_var_params = (fixed_param, trainable_param)

    return {
        "min_trace": (min_trace_params, min_trace_val),
        "max_trace": (max_trace_params, max_trace_val),
        "min_variance": (min_var_params, min_var_val),
        "max_variance": (max_var_params, max_var_val)
    }

def find_average_extreme_params(qfim_eigvals_dict):
    avg_trace_dict = {}
    avg_var_dict = {}
    nonzero_eigvals_dict = {}
    threshold = 1e-12
    for fixed_param, trainable_dict in qfim_eigvals_dict.items():
        traces = [np.sum(eigvals) for eigvals in trainable_dict.values()]
        variances = [np.var(eigvals[eigvals > threshold]) for eigvals in trainable_dict.values()]

        # nonzero_eigvals_dict[fixed_param] 
        avg_trace_dict[fixed_param] = np.mean(traces)
        avg_var_dict[fixed_param] = np.mean(variances)

    min_avg_trace_val = min(avg_trace_dict.values())
    max_avg_trace_val = max(avg_trace_dict.values())
    min_avg_var_val = min(avg_var_dict.values())
    max_avg_var_val = max(avg_var_dict.values())

    min_avg_trace_key = min(avg_trace_dict, key=avg_trace_dict.get)
    max_avg_trace_key = max(avg_trace_dict, key=avg_trace_dict.get)
    min_avg_var_key = min(avg_var_dict, key=avg_var_dict.get)
    max_avg_var_key = max(avg_var_dict, key=avg_var_dict.get)

    return {
        "avg_trace_dict": avg_trace_dict,
        "avg_var_dict": avg_var_dict,
        "min_avg_trace": (min_avg_trace_key, min_avg_trace_val),
        "max_avg_trace": (max_avg_trace_key, max_avg_trace_val),
        "min_avg_variance": (min_avg_var_key, min_avg_var_val),
        "max_avg_variance": (max_avg_var_key, max_avg_var_val)
    }

def find_range_extreme_params(qfim_eigvals_dict):
    trace_range_dict = {}
    var_range_dict = {}
    threshold = 1e-12
    for fixed_param, trainable_dict in qfim_eigvals_dict.items():
        traces = [np.sum(eigvals) for eigvals in trainable_dict.values()]
        variances = [np.var(eigvals[eigvals > threshold]) for eigvals in trainable_dict.values()]


        trace_range_dict[fixed_param] = max(traces) - min(traces)
        var_range_dict[fixed_param] = max(variances) - min(variances)

    min_trace_range_val = min(trace_range_dict.values())
    max_trace_range_val = max(trace_range_dict.values())
    min_var_range_val = min(var_range_dict.values())
    max_var_range_val = max(var_range_dict.values())

    min_trace_range_key = min(trace_range_dict, key=trace_range_dict.get)
    max_trace_range_key = max(trace_range_dict, key=trace_range_dict.get)
    min_var_range_key = min(var_range_dict, key=var_range_dict.get)
    max_var_range_key = max(var_range_dict, key=var_range_dict.get)

    return {
        "trace_range_dict": trace_range_dict,
        "var_range_dict": var_range_dict,
        "min_trace_range": (min_trace_range_key, min_trace_range_val),
        "max_trace_range": (max_trace_range_key, max_trace_range_val),
        "min_variance_range": (min_var_range_key, min_var_range_val),
        "max_variance_range": (max_var_range_key, max_var_range_val)
    }

def calculate_combined_score(avg_dict, range_dict, weight_avg=0.5, weight_range=0.5):
    """
    Calculate a combined score for each fixed parameter based on average and range values.
    The score is a weighted sum of normalized average and range values.
    """
    avg_values = list(avg_dict.values())
    range_values = list(range_dict.values())

    max_avg = max(avg_values)
    min_avg = min(avg_values)
    max_range = max(range_values)
    min_range = min(range_values)

    combined_scores = {}
    for key in avg_dict.keys():
        normalized_avg = (avg_dict[key] - min_avg) / (max_avg - min_avg)
        normalized_range = (range_dict[key] - min_range) / (max_range - min_range)
        combined_score = weight_avg * normalized_avg - weight_range * normalized_range
        combined_scores[key] = combined_score

    return combined_scores



# Example usage
N_ctrl = 1
Nr = 1
trotter_step = 1
state = 'GHZ'
kfactor = 1.0
base_path = f'/Users/sophieblock/QRCCapstone/QFIM_traced_final_results/analog_model_DQFIM/Nc_{N_ctrl}/{state}_state/'


file_path = Path(base_path) / f'Nr_{Nr}' / f'trotter_step_{trotter_step}/L_50' / 'data_2pi.pickle'



qfim_eigvals_dict, fixed_params_dict = get_qfim_eigvals(file_path)

jacobian_dict = get_jacobian(file_path)

if qfim_eigvals_dict is not None:
    results = find_extreme_params(qfim_eigvals_dict)
    print(f"Minimum trace: {results['min_trace']}")
    print(f"Maximum trace: {results['max_trace']}")
    print(f"Minimum variance: {results['min_variance']}")
    print(f"Maximum variance: {results['max_variance']}")

    avg_results = find_average_extreme_params(qfim_eigvals_dict)
    print(f"Minimum average trace: {avg_results['min_avg_trace']}")
    print(f"Maximum average trace: {avg_results['max_avg_trace']}")
    print(f"Minimum average variance: {avg_results['min_avg_variance']}")
    print(f"Maximum average variance: {avg_results['max_avg_variance']}")

    range_results = find_range_extreme_params(qfim_eigvals_dict)
    print(f"Minimum trace range: {range_results['min_trace_range']}")
    print(f"Maximum trace range: {range_results['max_trace_range']}")
    print(f"Minimum variance range: {range_results['min_variance_range']}")
    print(f"Maximum variance range: {range_results['max_variance_range']}")

    combined_trace_scores = calculate_combined_score(avg_results['avg_trace_dict'], range_results['trace_range_dict'])
    combined_variance_scores = calculate_combined_score(avg_results['avg_var_dict'], range_results['var_range_dict'])

    best_trace_param = max(combined_trace_scores, key=combined_trace_scores.get)
    best_variance_param = max(combined_variance_scores, key=combined_variance_scores.get)

    print(f"\nBest fixed parameter for trace optimization: {best_trace_param}")
    print(f"Best fixed parameter for variance optimization: {best_variance_param}")

    best_trace_param_min = min(combined_trace_scores, key=combined_trace_scores.get)
    best_variance_param_min = min(combined_variance_scores, key=combined_variance_scores.get)

    print(f"\nBest fixed parameter for trace optimization: {best_trace_param_min}")
    print(f"Best fixed parameter for variance optimization: {best_variance_param_min}")
    jacobian_variance_extremes = find_jacobian_variance_extremes(jacobian_dict)
    print(f"Minimum Jacobian variance: {jacobian_variance_extremes['min_jacobian_variance']}")
    print(f"Maximum Jacobian variance: {jacobian_variance_extremes['max_jacobian_variance']}")

def find_fixed_param_with_smallest_max_trace(qfim_eigvals_dict):
    smallest_max_trace = float('inf')
    best_fixed_param = None

    for fixed_param, trainable_dict in qfim_eigvals_dict.items():
        max_trace = float('-inf')
        for eigvals in trainable_dict.values():
            trace = np.sum(eigvals)
            if trace > max_trace:
                max_trace = trace
        
        if max_trace < smallest_max_trace:
            smallest_max_trace = max_trace
            best_fixed_param = fixed_param

    return best_fixed_param, smallest_max_trace
def find_fixed_param_with_largest_min_trace(qfim_eigvals_dict):
    largest_min_trace = float('-inf')
    best_fixed_param = None

    for fixed_param, trainable_dict in qfim_eigvals_dict.items():
        min_trace = float('inf')
        for eigvals in trainable_dict.values():
            trace = np.sum(eigvals)
            if trace < min_trace:
                min_trace = trace
        
        if min_trace > largest_min_trace:
            largest_min_trace = min_trace
            best_fixed_param = fixed_param


    return best_fixed_param, largest_min_trace

if qfim_eigvals_dict is not None:
    best_fixed_param_largest_min, largest_min_trace = find_fixed_param_with_largest_min_trace(qfim_eigvals_dict)
    print(f"\n\nBest fixed parameter set with largest minimum trace: {best_fixed_param_largest_min}")
    print(f"Largest minimum trace: {largest_min_trace}")

    best_fixed_param_smallest_max, smallest_max_trace = find_fixed_param_with_smallest_max_trace(qfim_eigvals_dict)
    print(f"Best fixed parameter set with smallest maximum trace: {best_fixed_param_smallest_max}")
    print(f"Smallest maximum trace: {smallest_max_trace}")

#/Users/sophieblock/QRCCapstone/QFIM_traced_final_results/analog_model_DQFIM/Nc_1/GHZ_state/Nr_1/trotter_step_1/L_50/data_2pi.pickle

Minimum trace: (('fixed_params0', 'test33'), Array(0.02194107, dtype=float32))
Maximum trace: (('fixed_params0', 'test15'), Array(163.14162, dtype=float32))
Minimum variance: (('fixed_params0', 'test33'), Array(0.00010695, dtype=float32))
Maximum variance: (('fixed_params0', 'test15'), Array(5906.65, dtype=float32))
Minimum average trace: ('fixed_params0', 22.230675)
Maximum average trace: ('fixed_params0', 22.230675)
Minimum average variance: ('fixed_params0', 420.5016)
Maximum average variance: ('fixed_params0', 420.5016)
Minimum trace range: ('fixed_params0', Array(163.11967, dtype=float32))
Maximum trace range: ('fixed_params0', Array(163.11967, dtype=float32))
Minimum variance range: ('fixed_params0', Array(5906.65, dtype=float32))
Maximum variance range: ('fixed_params0', Array(5906.65, dtype=float32))

Best fixed parameter for trace optimization: fixed_params0
Best fixed parameter for variance optimization: fixed_params0

Best fixed parameter for trace optimization: fixed_params

/var/folders/xx/w2tzbky134bg3g8mct87xb8c0000gn/T/ipykernel_47846/457945293.py:218: RuntimeWarning: invalid value encountered in scalar divide
  normalized_avg = (avg_dict[key] - min_avg) / (max_avg - min_avg)


In [6]:
def process_qfim_eigvals(qfim_eigvals_dict, threshold=1e-12, alpha=0.75):
    # Create a new dictionary to store processed values
    processed_qfim_dict = {}

    for fixed_params, test_results in qfim_eigvals_dict.items():
        avg_eigvals = []
        
        # Iterate over tests and collect eigenvalues
        for test, eigvals in test_results.items():
            # Apply threshold and compute mean along eigenvalue axis
            eigvals = np.array(eigvals)
            avg_eigvals.append(np.where(eigvals < threshold, 0, eigvals))
        
        # Compute the mean of all tests' averaged eigenvalues
        avg_qfim_eigvals = np.mean(avg_eigvals, axis=0)
        avg_trace = sum(avg_qfim_eigvals)
        # Apply power scaling
        power_scaled_avg_qfim_eigvals = avg_qfim_eigvals ** alpha

        power_scaled_avg_qfim_eigvals_nonzero = np.array([val for val in avg_qfim_eigvals if val > threshold]) ** alpha

        

        # Save both the averaged and power-scaled values
        processed_qfim_dict[fixed_params] = {
            'avg_qfim_eigvals': avg_qfim_eigvals,
            'power_scaled_avg_qfim_eigvals': power_scaled_avg_qfim_eigvals,
            'power_scaled_avg_qfim_eigvals_nonzero': power_scaled_avg_qfim_eigvals_nonzero,
            'var_power_scaled_avg_qfim_eigvals': np.var(power_scaled_avg_qfim_eigvals),

            'var_power_scaled_avg_qfim_eigvals_nonzero': np.var(power_scaled_avg_qfim_eigvals_nonzero),
            'avg_trace':avg_trace

        }

    return processed_qfim_dict
print(len(qfim_eigvals_dict['fixed_params0'].values()))
# Process the eigenvalues
processed_qfim_dict = process_qfim_eigvals(qfim_eigvals_dict)

# Convert to DataFrame for visualization
df_all = pd.DataFrame(processed_qfim_dict)

# Display DataFrame
df_all

100


,fixed_params0
avg_qfim_eigvals,"[0.0, 0.0, 0.0005340587, 0.06820663, 22.161942]"
avg_trace,22.230682
power_scaled_avg_qfim_eigvals,"[0.0, 0.0, 0.00351311, 0.13346578, 10.214235]"
power_scaled_avg_qfim_eigvals_nonzero,"[0.00351311, 0.13346578, 10.214235]"
var_power_scaled_avg_qfim_eigvals,16.58378
var_power_scaled_avg_qfim_eigvals_nonzero,22.877516


In [86]:
fixed_params_dict['fixed_params0'] # kfactor = 1.0

{'K_coef': Array([[-1.0051634 , -1.0372735 , -0.5005625 , -1.4021528 , -3.0429544 ],
        [-1.0372735 ,  0.97848654, -0.83255595, -1.7643881 , -2.7279024 ],
        [-0.5005625 , -0.83255595, -2.5485463 , -1.1928862 , -1.2101729 ],
        [-1.4021528 , -1.7643881 , -1.1928862 , -2.1490366 , -1.3298038 ],
        [-3.0429544 , -2.7279024 , -1.2101729 , -1.3298038 ,  1.1952485 ]],      dtype=float32)}

In [54]:
fixed_params_dict['fixed_params0'] # kfactor = 0.1

{'K_coef': Array([[-0.27346987,  0.09926751,  0.02546666, -0.136024  ],
        [ 0.09926751, -0.17941289,  0.01906796,  0.0279632 ],
        [ 0.02546666,  0.01906796, -0.21488734, -0.17762204],
        [-0.136024  ,  0.0279632 , -0.17762204,  0.10127006]],      dtype=float32)}

In [87]:
# Convert the processed dictionary into a DataFrame for easier operations
df_all = pd.DataFrame(processed_qfim_dict).T  # Transpose for easier access

# Now find the min/max keys for both var_power_scaled_avg_qfim_eigvals and var_power_scaled_avg_qfim_eigvals_nonzero
min_var_qfim_key = df_all['var_power_scaled_avg_qfim_eigvals'].idxmin(), df_all['var_power_scaled_avg_qfim_eigvals'].min()
max_var_qfim_key = df_all['var_power_scaled_avg_qfim_eigvals'].idxmax(), df_all['var_power_scaled_avg_qfim_eigvals'].max()

min_var_qfim_nonzero_key = df_all['var_power_scaled_avg_qfim_eigvals_nonzero'].idxmin(), df_all['var_power_scaled_avg_qfim_eigvals_nonzero'].min()
max_var_qfim_nonzero_key = df_all['var_power_scaled_avg_qfim_eigvals_nonzero'].idxmax(), df_all['var_power_scaled_avg_qfim_eigvals_nonzero'].max()

# Nicely formatted output
print("Results for 'var_power_scaled_avg_qfim_eigvals':")
print(f"Minimum variance found for fixed parameter set: {min_var_qfim_key[0]}, with value: {min_var_qfim_key[1]:.5f}")
print(f"Maximum variance found for fixed parameter set: {max_var_qfim_key[0]}, with value: {max_var_qfim_key[1]:.5f}")

print("\nResults for 'var_power_scaled_avg_qfim_eigvals_nonzero':")
print(f"Minimum variance found for fixed parameter set: {min_var_qfim_nonzero_key[0]}, with value: {min_var_qfim_nonzero_key[1]:.5f}")
print(f"Maximum variance found for fixed parameter set: {max_var_qfim_nonzero_key[0]}, with value: {max_var_qfim_nonzero_key[1]:.5f}")



min_trace_qfim_nonzero_key = df_all['avg_trace'].idxmin(), df_all['avg_trace'].min()
max_trace_qfim_nonzero_key = df_all['avg_trace'].idxmax(), df_all['avg_trace'].max()


print("\nResults for 'minimum trace':")
print(f"Minimum variance found for fixed parameter set: {min_trace_qfim_nonzero_key[0]}, with value: {min_trace_qfim_nonzero_key[1]:.5f}")
print(f"Maximum variance found for fixed parameter set: {max_trace_qfim_nonzero_key[0]}, with value: {max_trace_qfim_nonzero_key[1]:.5f}")


Results for 'var_power_scaled_avg_qfim_eigvals':
Minimum variance found for fixed parameter set: fixed_params1, with value: 43.33490
Maximum variance found for fixed parameter set: fixed_params2, with value: 161.43019

Results for 'var_power_scaled_avg_qfim_eigvals_nonzero':
Minimum variance found for fixed parameter set: fixed_params1, with value: 123.60330
Maximum variance found for fixed parameter set: fixed_params2, with value: 481.56686

Results for 'minimum trace':
Minimum variance found for fixed parameter set: fixed_params1, with value: 75.61408
Maximum variance found for fixed parameter set: fixed_params2, with value: 177.05310


In [88]:
qfim_variances_by_fixedparam_dict = {}

for fp,eigs in qfim_eigvals_dict.items():

    qfim_variances_by_fixedparam_dict.setdefault(fp, {}).setdefault('avg_eigs', []).extend(eigs)
    # qfim_variances_by_fixedparam_dict.setdefault(fp, {}).setdefault('avg_eigs', []).extend(eigs)
df_all = pd.DataFrame(qfim_variances_by_fixedparam_dict)

In [9]:
df_all = pd.DataFrame(qfim_variances_by_fixedparam_dict)


In [3]:
threshold = 1e-14

def process_params(params):
    """Process a dictionary of parameters, converting all numeric values to floats or lists of floats."""
    #print(params)
    return {key: convert_to_float(value) for key, value in params.items()}
def convert_to_float(value):
    """Convert NumPy arrays or other numeric types to float or list of floats."""
    if isinstance(value, np.ndarray):
        # Convert NumPy array to list of floats
        return [float(f"{x:.13f}") for x in value]
    elif isinstance(value, (float, int, np.float32, np.float64, np.int32, np.int64)):
        # Convert single numeric value to float
        return float(f"{value:.13f}")
    else:
        # Return the value as-is if it's not a recognized numeric type
        return value
    
def convert_to_hashable(value):
    """Convert unhashable types (e.g., NumPy arrays) to hashable types (e.g., tuples)."""
    if isinstance(value, np.ndarray):
        return tuple(value.tolist())  # Convert arrays to tuples
    elif isinstance(value, list):
        return tuple(value)  # Convert lists to tuples
    else:
        return value

def process_params_for_hashing(params):
    """Process a dictionary of parameters, converting all values to hashable types."""
    return {key: convert_to_hashable(value) for key, value in params.items()}

some_results = {}


def find_max_variance_params(file_path,track):
    """
    Load data from a pickle file and find the parameters with the maximum mean variance of the qfim eigenvalues.

    Parameters:
    - file_path: Path object or str, the path to the pickle file.
    - metric: str, the metric to compare, e.g., 'mean_var_eigval' for mean variance of the qfim eigenvalues.

    Returns:
    Tuple of dictionaries for the fixed and trainable parameters with the maximum value of the specified metric.
    """
    # Ensure file_path is a Path object
    file_path = Path(file_path) if not isinstance(file_path, Path) else file_path
    min_trace = np.inf
    if not file_path.exists():
        print(f"File {file_path} does not exist.")
        return None

    with open(file_path, 'rb') as f:
        df = pickle.load(f)
    
    max_result = -np.inf
    min_result = np.inf
    max_trace = -np.inf
    min_trace = np.inf
    max_trace_params = {}
    min_trace_params = {}
    max_trained_params = {}
    
    min_trained_params = {}
    max_fixed_params = None
    max_trained_params = {}
    min_fixed_params = None
    
    
    qfim_eigvals_dict = {}
    fixed_params_values_dict = {}
    
    for i, fixed_params_dict in enumerate(df.keys()):
        for k,test in enumerate(df[fixed_params_dict].keys()):
            #if  test == 'test13' or test == 'test14' or test == 'test11':
            #    continue
            #if test == 'test25' or test == 'test95':
            #    continue
            results = df[fixed_params_dict][test]
            if 'qfim_eigvals' in results:
                qfim_eigvals = results['qfim_eigvals']
                nonzero_eigvals = qfim_eigvals[qfim_eigvals > threshold]
                res = np.var(nonzero_eigvals)
                trace = np.sum(qfim_eigvals)
                #print(test,trace, min_trace)
                
                if trace < min_trace and fixed_params_dict == track:
                    min_trace = trace
                    
                    min_trainableparams =  convert_to_float(results.get('trainable_params', []))
                    min_trainableparams = '[' + ', '.join([f"{x:.16f}" for x in min_trainableparams]) + ']'
                    min_trainableparams_eigvals = '[' + ', '.join([f"{x:.16f}" for x in qfim_eigvals]) + ']'
                    min_trace_params[fixed_params_dict] = {test:{"trace":trace,"params":min_trainableparams,
                                                                   "eigvals":min_trainableparams_eigvals}} 
                if trace > max_trace and fixed_params_dict == track:
                    max_trace = trace
                    print(test, max_trace)
                    max_trainableparams =  convert_to_float(results.get('trainable_params', []))
                    max_trainableparams = '[' + ', '.join([f"{x:.16f}" for x in max_trainableparams]) + ']'
                    max_trainableparams_eigvals = '[' + ', '.join([f"{x:.16f}" for x in qfim_eigvals]) + ']'
                    max_trace_params[fixed_params_dict] =  {test:{"trace":trace,"params":max_trainableparams,
                                                                   "eigvals":max_trainableparams_eigvals}} 
                if res < min_result and fixed_params_dict == track:
                    min_result = res
                    
                    min_trainableparams =  convert_to_float(results.get('trainable_params', []))
                    min_trainableparams = '[' + ', '.join([f"{x:.16f}" for x in min_trainableparams]) + ']'
                    min_trainableparams_eigvals = '[' + ', '.join([f"{x:.16f}" for x in qfim_eigvals]) + ']'
                    min_trained_params[fixed_params_dict] = {test:{"var":res,"params":min_trainableparams,
                                                                   "eigvals":min_trainableparams_eigvals}} 
                if res > max_result and fixed_params_dict == track:
                    max_result = res
                    
                    max_trainableparams =  convert_to_float(results.get('trainable_params', []))
                    max_trainableparams = '[' + ', '.join([f"{x:.16f}" for x in max_trainableparams]) + ']'
                    max_trainableparams_eigvals = '[' + ', '.join([f"{x:.16f}" for x in qfim_eigvals]) + ']'
                    max_trained_params[fixed_params_dict] =  {test:{"var":res,"params":max_trainableparams,
                                                                   "eigvals":max_trainableparams_eigvals}} 
                    
                if fixed_params_dict not in qfim_eigvals_dict:
                    qfim_eigvals_dict[fixed_params_dict] = []
                
                qfim_eigvals_dict[fixed_params_dict].append(qfim_eigvals)
                if fixed_params_dict not in fixed_params_values_dict:
                    fixed_params_values_dict[fixed_params_dict] = {}
                
                fixed_params_values_dict[fixed_params_dict]['fixed_params'] = results.get('fixed_params', {})
    print(k)
    return qfim_eigvals_dict, fixed_params_values_dict,min_trace,max_result,max_trace_params,min_trace_params,max_trained_params,min_trained_params


param_set_key = 'fixed_params0'
qfim_eigvals_dict, fixed_params_values_dict,min_trace,max_trace,max_trace_params,min_trace_params,max_trained_params,min_trained_params = find_max_variance_params(file_path,param_set_key)



#print("\nFixed Parameters Values Dictionary:")
#for k, v in fixed_params_values_dict.items():
#    print(f"{k}: {v}")
print(min_trained_params)
print(max_trained_params)
print(min_trace_params)
print(max_trace_params)

param_set_key = 'fixed_params1'
qfim_eigvals_dict, fixed_params_values_dict,min_trace,max_trace,max_trace_params,min_trace_params,max_trained_params,min_trained_params = find_max_variance_params(file_path,param_set_key)



#print("\nFixed Parameters Values Dictionary:")
#for k, v in fixed_params_values_dict.items():
#    print(f"{k}: {v}")
print(min_trained_params)
print(max_trained_params)
print(min_trace_params)
print(max_trace_params)


param_set_key = 'fixed_params2'
qfim_eigvals_dict, fixed_params_values_dict,min_trace,max_trace,max_trace_params,min_trace_params,max_trained_params,min_trained_params = find_max_variance_params(file_path,param_set_key)



#print("\nFixed Parameters Values Dictionary:")
#for k, v in fixed_params_values_dict.items():
#    print(f"{k}: {v}")
print(min_trained_params)
print(max_trained_params)
print(min_trace_params)
print(max_trace_params)


param_set_key = 'fixed_params19'
qfim_eigvals_dict, fixed_params_values_dict,min_trace,max_trace,max_trace_params,min_trace_params,max_trained_params,min_trained_params = find_max_variance_params(file_path,param_set_key)



#print("\nFixed Parameters Values Dictionary:")
#for k, v in fixed_params_values_dict.items():
#    print(f"{k}: {v}")
print(min_trained_params)
print(max_trained_params)
print(min_trace_params)
print(max_trace_params)



test0 1.7730381
test1 5.9445853
test3 9.102309
test4 106.41627
test11 155.63834
test30 186.38506
test31 196.57744
test88 212.26529
test104 214.21696
test146 338.0931
149
{'fixed_params0': {'test50': {'var': Array(7.0880833e-06, dtype=float32), 'params': '[0.6913065314292908, 0.0089570693671703, 2.9914927482604980, 5.9865536689758301, 4.4180288314819336, 0.1592517644166946, 0.3302091956138611]', 'eigvals': '[-0.0000000000000000, -0.0000000000000000, 0.0000000000000000, 0.0000000000000179, 0.0000274282301689, 0.0027888638433069, 0.0065026539377868]'}}}
{'fixed_params0': {'test146': {'var': Array(21432.34, dtype=float32), 'params': '[0.2009621411561966, 0.3206783235073090, 0.7736065983772278, 1.0596547126770020, 0.1131188422441483, 4.7357854843139648, 5.1343908309936523]', 'eigvals': '[-0.0000000000000000, 0.0000000000000000, 0.0000000000000001, 0.0000000000001011, 0.0002811973972712, 0.0009670032886788, 338.0918579101562500]'}}}
{'fixed_params0': {'test50': {'trace': Array(0.00931895, dt

In [37]:

# Example usage
N_ctrl = 1
Nr = 1
trotter_step = 1
state = 'GHZ'
kfactor = 1.0
# base_path = f'/Users/sophieblock/QRCCapstone/QFIM_traced_trainable_global/analog_model_DQFIM/Nc_{N_ctrl}/{state}_state/'


# file_path = Path(base_path) / f'Nr_{Nr}' / f'trotter_step_{trotter_step}/{kfactor}K' / 'data.pickle'



with open(file_path, 'rb') as f:
    all_tests_data = pickle.load(f)

#print(all_tests_data)
params_sorted_by_qfim_trace = {}
params_sorted_by_qfim_eigvals = {}
params_sorted_by_qfim_var,params_sorted_by_qfim_nonzero_var = {},{}
params_sorted_by_qfim_mineig = {}
params_sorted_by_qfim_qfi_entropy,params_sorted_by_qfim_entropy,params_sorted_by_cond = {},{},{}

key_and_params = {}
nnz = {}
fixed_params_dict = 'fixed_params0'
#if fixed_params_dict == 'fixed_params2':
#    continue
threshold = 1e-12
test_data = all_tests_data[fixed_params_dict]
for trainable_params_dict in test_data.keys():
    #print(trainable_params_dict)
    results = test_data[trainable_params_dict]
    
    qfi_eigvals = results['qfim_eigvals']
    nonzero_eigvals = qfi_eigvals[qfi_eigvals > threshold]
    #print(trainable_params_dict,entropy)
    trace = np.sum(qfi_eigvals)
    min_eigval = min(qfi_eigvals)
    variance = np.var(qfi_eigvals)
    variance_nonzero = np.var(nonzero_eigvals)
    params_sorted_by_qfim_trace[trainable_params_dict] = float(trace)
    params_sorted_by_qfim_var[trainable_params_dict] = float(variance)
    params_sorted_by_qfim_nonzero_var[trainable_params_dict] = float(variance_nonzero)
    params_sorted_by_qfim_mineig[trainable_params_dict] = float(min_eigval)
    dqfim_entropy = compute_qfim_entropy(qfi_eigvals)
    dqfim_condition = np.max(qfi_eigvals) / np.min(nonzero_eigvals)
    params_sorted_by_cond[trainable_params_dict] =  dqfim_condition
    params_sorted_by_qfim_qfi_entropy[trainable_params_dict] =  dqfim_entropy


    key_and_params[trainable_params_dict] = results['trainable_params']
    
    params_sorted_by_qfim_eigvals[trainable_params_dict] =  qfi_eigvals
    nnz[trainable_params_dict] = len(nonzero_eigvals)
    params_sorted_by_qfim_entropy[trainable_params_dict] =  float(np.mean(results['entropies']))

# print("params_sorted_by_qfim_entropy: ",params_sorted_by_qfim_entropy)
# print("params_sorted_by_qfim_trace: ",params_sorted_by_qfim_trace)
# print("params_sorted_by_qfim_var: ",params_sorted_by_qfim_var)
number_of_tests = len(test_data)

sorted_qfim_entropy = sorted(params_sorted_by_qfim_entropy.items(), key=lambda x: round(float(x[1]), 6))

sorted_qfim_trace = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: float(x[1]))

sorted_qfim_var = sorted(params_sorted_by_qfim_var.items(), key=lambda x: round(float(x[1]),5))

sorted_qfim_var_nonzero = sorted(params_sorted_by_qfim_nonzero_var.items(), key=lambda x: round(float(x[1]),5))

sorted_qfim_qfi_entropy = sorted(params_sorted_by_qfim_qfi_entropy.items(), key=lambda x: round(float(x[1]), 6))

# Calculate the number of parameters to select
number_of_tests = len(test_data)

# Convert arrays to scalars for sorting
#sorted_qfim_trace = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: x[1].item() if isinstance(x[1], np.ndarray) else x[1])

print(f"sorted_qfim_trace: {sorted_qfim_trace}")
print(f"sorted_qfim_trace[::-1]: {sorted_qfim_trace[::-1]}\n")

print(f"sorted_qfim_var: {sorted_qfim_var}")
print(f"sorted_qfim_var[::-1]: {sorted_qfim_var[::-1]}\n")


print(f"sorted_qfim_entropy: {sorted_qfim_entropy}")
print(f"sorted_qfim_entropy[::-1]: {sorted_qfim_entropy[::-1]}\n")



print(f"sorted_qfim_entropy: {sorted_qfim_qfi_entropy}")
print(f"sorted_qfim_entropy[::-1]: {sorted_qfim_qfi_entropy[::-1]}\n")

# Calculate the number of parameters to select
num_to_select = len(sorted_qfim_trace) // 10

# Determine interval size
interval_size = len(sorted_qfim_trace) / num_to_select

# Select parameters based on intervals
selected_params = []
selected_params_with_vals = []

for i in range(num_to_select):
    start_idx = int(i * interval_size)
    end_idx = int((i + 1) * interval_size)
    interval = sorted_qfim_trace[start_idx:end_idx]
    midpoint = (interval[0][1] + interval[-1][1]) / 2
    closest_param = min(interval, key=lambda x: abs(x[1] - midpoint))
    selected_params.append(closest_param[0])
    selected_params_with_vals.append(closest_param)

print("Selected Parameters:", selected_params)
print("Selected Parameters with Values:", selected_params_with_vals)


smallest_key = sorted_qfim_trace[0][0]
print(f"min: {smallest_key}")
print(f"max: {sorted_qfim_trace[-1][0]}")

for i in range(1,6):
    key = sorted_qfim_trace[-i][0]


    if key not in selected_params:
        print(f"adding {key}...")
        selected_params.append(key)
        selected_params_with_vals.append((key,params_sorted_by_qfim_trace[key]))


for i in range(0,4):
    key = sorted_qfim_trace[i][0]


    if key not in selected_params:
        print(f"adding min {key}...")
        selected_params.append(key)
        selected_params_with_vals.append((key,params_sorted_by_qfim_trace[key]))


if smallest_key not in selected_params:
    selected_params.append(smallest_key)
    selected_params_with_vals.append((smallest_key,params_sorted_by_qfim_trace[smallest_key]))

print("Selected Parameters:", selected_params)

selected_params_with_vals = sorted(selected_params_with_vals, key=lambda x: float(x[1]))

print("Selected Parameters with Values:", selected_params_with_vals)


# Select parameters based on intervals
selected_params_vars = []
selected_params_with_vals_vars = []

for i in range(num_to_select):
    start_idx = int(i * interval_size)
    end_idx = int((i + 1) * interval_size)
    interval = sorted_qfim_var[start_idx:end_idx]
    midpoint = (interval[0][1] + interval[-1][1]) / 2
    closest_param = min(interval, key=lambda x: abs(x[1] - midpoint))
    selected_params_vars.append(closest_param[0])
    selected_params_with_vals_vars.append(closest_param)
    
print("Variances:",len(selected_params_with_vals_vars), selected_params_with_vals_vars)

sorted_qfim_trace: [('test33', 0.021941067650914192), ('test89', 0.05461035296320915), ('test80', 0.05808810517191887), ('test91', 0.06624511629343033), ('test75', 0.07214505225419998), ('test60', 0.08045545965433121), ('test28', 0.13817963004112244), ('test20', 0.15757901966571808), ('test62', 0.17422173917293549), ('test52', 0.18709862232208252), ('test76', 0.20058053731918335), ('test72', 0.25965943932533264), ('test64', 0.31362318992614746), ('test21', 0.3678807020187378), ('test13', 0.3955928385257721), ('test2', 0.4843137562274933), ('test9', 0.5132178068161011), ('test48', 0.5394481420516968), ('test12', 0.5511218905448914), ('test86', 0.5710611343383789), ('test92', 0.5981006622314453), ('test4', 0.6270114183425903), ('test17', 0.6413464546203613), ('test19', 0.7992272973060608), ('test35', 0.8233681321144104), ('test11', 1.0734143257141113), ('test39', 1.1081702709197998), ('test3', 1.1391879320144653), ('test67', 1.2423852682113647), ('test30', 1.2503114938735962), ('test54',

In [30]:
def sort_top_traces_by_entropy(params_sorted_by_qfim_trace, params_sorted_by_qfim_entropy, top_n=10):
    """
    Identify the top N largest trace entries (in magnitude) and sort them by their entropy from smallest to largest.
    
    Args:
        params_sorted_by_qfim_trace (dict): Dictionary with trace values.
        params_sorted_by_qfim_entropy (dict): Dictionary with entropy values.
        top_n (int): Number of largest traces to consider.
    
    Returns:
        List of tuples with the top N traces sorted by entropy.
    """
    # Step 1: Sort the trace values by magnitude, largest to smallest
    sorted_by_trace_magnitude = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: abs(x[1]), reverse=True)
    
    # Step 2: Get the top N entries with the largest trace magnitudes
    top_n_traces = sorted_by_trace_magnitude[:top_n]
    
    # Step 3: Sort the top N entries by their corresponding entropy values (smallest to largest)
    sorted_top_n_by_entropy = sorted(top_n_traces, key=lambda x: params_sorted_by_qfim_entropy[x[0]])
    
    return sorted_top_n_by_entropy

# Example usage:
top_n_sorted_by_entropy = sort_top_traces_by_entropy(params_sorted_by_qfim_trace, params_sorted_by_qfim_entropy, top_n=10)

# Print results
print("Top 10 largest trace values, sorted by entropy:")
for param, trace in top_n_sorted_by_entropy:
    print(f"Parameter: {param}, Trace: {trace:.2e}, Entropy: {params_sorted_by_qfim_entropy[param]:.2e}")

    # Example usage:
top_n_sorted_by_entropy = sort_top_traces_by_entropy(params_sorted_by_qfim_var, params_sorted_by_qfim_entropy, top_n=10)

# Print results
print("\nTop 10 largest var values, sorted by entropy:")
for param, var in top_n_sorted_by_entropy:
    print(f"Parameter: {param}, Var: {var:.2e}, Trace: {params_sorted_by_qfim_trace[param]:.2e},  Entropy: {params_sorted_by_qfim_entropy[param]:.5f}")



Top 10 largest trace values, sorted by entropy:
Parameter: test57, Trace: 1.37e+02, Entropy: 2.02e-01
Parameter: test6, Trace: 1.43e+02, Entropy: 2.64e-01
Parameter: test1, Trace: 1.58e+02, Entropy: 2.67e-01
Parameter: test36, Trace: 9.59e+01, Entropy: 2.76e-01
Parameter: test87, Trace: 9.15e+01, Entropy: 2.98e-01
Parameter: test73, Trace: 1.07e+02, Entropy: 3.79e-01
Parameter: test24, Trace: 9.07e+01, Entropy: 4.14e-01
Parameter: test58, Trace: 1.01e+02, Entropy: 4.51e-01
Parameter: test7, Trace: 1.20e+02, Entropy: 4.73e-01
Parameter: test15, Trace: 1.63e+02, Entropy: 5.13e-01

Top 10 largest var values, sorted by entropy:
Parameter: test57, Var: 3.01e+03, Trace: 1.37e+02,  Entropy: 0.20237
Parameter: test6, Var: 3.26e+03, Trace: 1.43e+02,  Entropy: 0.26407
Parameter: test1, Var: 3.98e+03, Trace: 1.58e+02,  Entropy: 0.26694
Parameter: test36, Var: 1.47e+03, Trace: 9.59e+01,  Entropy: 0.27570
Parameter: test87, Var: 1.34e+03, Trace: 9.15e+01,  Entropy: 0.29850
Parameter: test73, Var: 1

In [44]:
def sort_top_reverse(sorted_A, sorted_B, sorted_entropy, top_n=10, entropy_threshold=float('inf'), trace_threshold=0.0, var_threshold=100000):
    """
    Sort the top N entries based on the magnitude of values in sorted_A (reverse).
    Filter results where the corresponding entropy is below the entropy_threshold
    and trace is above the trace_threshold.
    
    Args:
        sorted_A (dict): Dictionary to sort by (e.g., variance).
        sorted_B (dict): Dictionary used for secondary sorting (e.g., trace).
        sorted_entropy (dict): Dictionary with entropy values.
        top_n (int): Number of top entries to select.
        entropy_threshold (float): Maximum allowed entropy value for entries to be included.
        trace_threshold (float): Minimum allowed trace value for entries to be included.
    
    Returns:
        List of tuples containing sorted results based on sorted_A.
    """
    # Step 1: Sort the trace values by magnitude, smallest to largest
    sorted_by_trace_magnitude = sorted(sorted_A.items(), key=lambda x: abs(x[1]))
    
    # Step 2: Filter based on entropy and trace thresholds
    filtered_by_entropy_and_trace = [(param, value) for param, value in sorted_by_trace_magnitude
                                     if sorted_entropy[param] <= entropy_threshold and sorted_A[param] > trace_threshold]
    # Step 2: Filter based on entropy and trace thresholds
    filtered_by_variance = [(param, value) for param, value in filtered_by_entropy_and_trace
                                     if sorted_B[param] < var_threshold]
    
    # Step 3: Get the top N entries from the filtered results
    top_n_traces = filtered_by_variance[:]
    
    # Step 4: Sort the top N entries by their corresponding trace values in reverse
    sorted_top_n = sorted(top_n_traces, key=lambda x: sorted_entropy[x[0]], reverse=True)
    
    return sorted_top_n

# Example usage:
top_n_sorted_by_entropy_and_trace = sort_top_reverse(
    params_sorted_by_qfim_trace, 
    params_sorted_by_qfim_nonzero_var, 
    params_sorted_by_qfim_qfi_entropy, 
    top_n=100, 
    entropy_threshold=0.1,  # Specify your entropy threshold here
    trace_threshold=20.0    # Specify your trace threshold here
)

# Print results
print("\nTop 30 largest var values, sorted by trace (filtered by entropy and trace thresholds):")
for param, trace in top_n_sorted_by_entropy_and_trace:
    print(f"Parameter: {param}, Trace: {trace:2f}, Var: {params_sorted_by_qfim_nonzero_var[param]:4f}, VNEntropy: {params_sorted_by_qfim_entropy[param]:5f}, Entropy: {params_sorted_by_qfim_qfi_entropy[param]:5f}")



Top 30 largest var values, sorted by trace (filtered by entropy and trace thresholds):
Parameter: test65, Trace: 20.154032, Var: 87.649567, VNEntropy: 0.288723, Entropy: 0.055032
Parameter: test96, Trace: 26.469761, Var: 153.963531, VNEntropy: 0.074293, Entropy: 0.024588
Parameter: test34, Trace: 21.007153, Var: 97.458855, VNEntropy: 0.249878, Entropy: 0.015408
Parameter: test40, Trace: 39.540730, Var: 345.247955, VNEntropy: 0.141668, Entropy: 0.015087
Parameter: test29, Trace: 37.336380, Var: 308.066040, VNEntropy: 0.352867, Entropy: 0.013476
Parameter: test36, Trace: 95.900406, Var: 2033.172485, VNEntropy: 0.275701, Entropy: 0.012840
Parameter: test32, Trace: 51.219646, Var: 580.480164, VNEntropy: 0.356798, Entropy: 0.010845
Parameter: test8, Trace: 82.475769, Var: 1506.078125, VNEntropy: 0.238464, Entropy: 0.009432
Parameter: test98, Trace: 21.533052, Var: 102.710175, VNEntropy: 0.263457, Entropy: 0.008512
Parameter: test58, Trace: 101.300934, Var: 2273.818359, VNEntropy: 0.450807,

In [17]:
def sort_min_traces_by_entropy(sorted_A, sorted_B, sorted_entropy, top_n=10, entropy_threshold=0.0, trace_threshold=float('inf')):
    """
    Find the top N entries with the smallest trace values, filtering by minimum entropy and maximum trace thresholds.
    
    Args:
        sorted_A (dict): Dictionary to sort by (e.g., variance).
        sorted_B (dict): Dictionary with trace values.
        sorted_entropy (dict): Dictionary with entropy values.
        top_n (int): Number of top entries to select.
        entropy_threshold (float): Minimum allowed entropy value for entries to be included.
        trace_threshold (float): Maximum allowed trace value for entries to be included.
    
    Returns:
        List of tuples containing sorted results based on sorted_A.
    """
    # Step 1: Sort the trace values by magnitude, smallest to largest
    sorted_by_trace_magnitude = sorted(sorted_A.items(), key=lambda x: abs(x[1]))
    
    # Step 2: Filter based on the trace and entropy thresholds
    filtered_by_entropy_and_trace = [(param, value) for param, value in sorted_by_trace_magnitude
                                     if sorted_entropy[param] >= entropy_threshold and sorted_A[param] < trace_threshold]
    
    # Step 3: Get the top N entries from the filtered results
    top_n_traces = filtered_by_entropy_and_trace[:]
    
    # Step 4: Sort the top N entries by their corresponding trace values in increasing order
    sorted_top_n = sorted(top_n_traces, key=lambda x: sorted_B[x[0]])
    
    return sorted_top_n

# Example usage:
top_n_sorted_by_min_trace_and_entropy = sort_min_traces_by_entropy(
    params_sorted_by_qfim_trace, 
    params_sorted_by_qfim_nonzero_var, 
    params_sorted_by_qfim_entropy, 
    top_n=100, 
    entropy_threshold=0.01,  # Specify your minimum entropy threshold here
    trace_threshold=1.0    # Specify your maximum trace threshold here
)

# Print results
print("\nTop 30 smallest var values, sorted by trace (filtered by minimum entropy and maximum trace thresholds):")
for param, trace in top_n_sorted_by_min_trace_and_entropy:
    print(f"Parameter: {param}, Trace: {trace:.4f}, Var: {params_sorted_by_qfim_var[param]:.4f}, Entropy: {params_sorted_by_qfim_entropy[param]:5f}")



Top 30 smallest var values, sorted by trace (filtered by minimum entropy and maximum trace thresholds):
Parameter: test80, Trace: 0.0581, Var: 0.0005, Entropy: 0.111850
Parameter: test91, Trace: 0.0662, Var: 0.0007, Entropy: 0.168949
Parameter: test75, Trace: 0.0721, Var: 0.0008, Entropy: 0.102197
Parameter: test60, Trace: 0.0805, Var: 0.0009, Entropy: 0.330408
Parameter: test20, Trace: 0.1576, Var: 0.0022, Entropy: 0.401880
Parameter: test28, Trace: 0.1382, Var: 0.0022, Entropy: 0.323388
Parameter: test62, Trace: 0.1742, Var: 0.0047, Entropy: 0.179263
Parameter: test52, Trace: 0.1871, Var: 0.0050, Entropy: 0.324793
Parameter: test76, Trace: 0.2006, Var: 0.0064, Entropy: 0.114878
Parameter: test64, Trace: 0.3136, Var: 0.0099, Entropy: 0.301221
Parameter: test72, Trace: 0.2597, Var: 0.0102, Entropy: 0.033527
Parameter: test2, Trace: 0.4843, Var: 0.0173, Entropy: 0.180459
Parameter: test92, Trace: 0.5981, Var: 0.0231, Entropy: 0.229511
Parameter: test21, Trace: 0.3679, Var: 0.0185, Entr

In [41]:
tests = ['test22','test50','test44','test125','test121', 'test53','test31','test94', 'test23'] # fp 19
tests = ['test22','test50','test44','test125','test121', 'test53','test31','test94', 'test23'] 
# tests = ['test22','test50','test44','test2','test104','test62','test69', 'test88','test146','test138', 'test63','test94'] 
tests =['test33','test80','test60','test53','test66', 'test96','test42','test43','test57', 'test41']
# tests = ['test33','test89','test80', 'test60','test20']

for test in tests:
    print(f"\n{test} (nnz: {nnz[test]}):")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]}")
    print(f"Var: {params_sorted_by_qfim_var[test]}")
    print(f"Var (nonzero included): {params_sorted_by_qfim_nonzero_var[test]}")
    # print(f"avg eig: {np.mean(params_sorted_by_qfim_eigvals[test]):.5e} +/- {np.std(params_sorted_by_qfim_eigvals[test]):.5e}")
    print(f'vn entropy: {params_sorted_by_qfim_entropy[test]}')
    print(f'entropy: {params_sorted_by_qfim_qfi_entropy[test]}')


test33 (nnz: 3):
Trace: 0.021941067650914192
Var: 7.700918649788946e-05
Var (nonzero included): 0.00010695261153159663
vn entropy: 0.0035675738472491503
entropy: 0.0009047804633155465

test80 (nnz: 3):
Trace: 0.05808810517191887
Var: 0.0005196902202442288
Var (nonzero included): 0.0007161846733652055
vn entropy: 0.11185044795274734
entropy: 0.08013878762722015

test60 (nnz: 3):
Trace: 0.08045545965433121
Var: 0.0009368705796077847
Var (nonzero included): 0.0012737585930153728
vn entropy: 0.3304080069065094
entropy: 0.1692543774843216

test53 (nnz: 3):
Trace: 71.34952545166016
Var: 813.6488037109375
Var (nonzero included): 1129.8255615234375
vn entropy: 0.1615292727947235
entropy: 0.0037520721089094877

test66 (nnz: 3):
Trace: 52.115562438964844
Var: 434.01605224609375
Var (nonzero included): 602.647705078125
vn entropy: 0.13088466227054596
entropy: 0.0043492186814546585

test96 (nnz: 3):
Trace: 26.46976089477539
Var: 111.06207275390625
Var (nonzero included): 153.96353149414062
vn ent

In [13]:
tests = ['test62','test69','test109'] # BAD?
tests = ['test22','test63','test123','test94','test104', 'test23','test33'] # good?
for test in tests:
    print(f"\n{test} (nnz: {nnz[test]}):")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]}")
    print(f"Var: {params_sorted_by_qfim_var[test]}")
    print(f"Var (nonzero included): {params_sorted_by_qfim_nonzero_var[test]}")
    print(f"avg eig: {np.mean(params_sorted_by_qfim_eigvals[test]):.5e} +/- {np.std(params_sorted_by_qfim_eigvals[test]):.5e}")
    print(f'\entropy: {params_sorted_by_qfim_entropy[test]}')



test22 (nnz: 3):
Trace: 0.009813008829951286
Var: 6.889848009450361e-06
Var (nonzero included): 9.96233484329423e-06
avg eig: 1.40186e-03 +/- 2.62485e-03
\entropy: 0.0006349763716571033

test63 (nnz: 3):
Trace: 193.7422637939453
Var: 4596.16796875
Var (nonzero included): 8341.150390625
avg eig: 2.76775e+01 +/- 6.77950e+01
\entropy: 0.28797560930252075

test123 (nnz: 3):
Trace: 0.9743103981018066
Var: 0.08995901793241501
Var (nonzero included): 0.14963258802890778
avg eig: 1.39187e-01 +/- 2.99932e-01
\entropy: 0.3273220956325531

test94 (nnz: 3):
Trace: 78.34285736083984
Var: 751.4769287109375
Var (nonzero included): 1363.7572021484375
avg eig: 1.11918e+01 +/- 2.74131e+01
\entropy: 0.21624116599559784

test104 (nnz: 3):
Trace: 214.2169647216797
Var: 5618.85546875
Var (nonzero included): 10197.08203125
avg eig: 3.06024e+01 +/- 7.49590e+01
\entropy: 0.37198007106781006

test23 (nnz: 3):
Trace: 80.91996002197266
Var: 801.6774291992188
Var (nonzero included): 1454.8323974609375
avg eig: 1.

In [84]:
tests = ['test84','test61','test76','test178','test107','test87','test23','test184']
# test0 2421.1675
# test5 2652.6914
# test9 3107.1506
# test29 3127.0017
# test31 3957.8772
# test81 4430.729
# test116 4449.634
# tests =['test53','test42','test25','test38','test177', 'test78']
for test in tests:
    print(f"\n{test} (nnz: {nnz[test]}):")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]}")
    print(f"Var: {params_sorted_by_qfim_var[test]}")
    print(f"Var (nonzero included): {params_sorted_by_qfim_nonzero_var[test]}")
    print(f"avg eig: {np.mean(params_sorted_by_qfim_eigvals[test]):.5e} +/- {np.std(params_sorted_by_qfim_eigvals[test]):.5e}")
    print(f'\entropy: {params_sorted_by_qfim_entropy[test]}')




test84 (nnz: 15):
Trace: 634.0263671875
Var: 12919.1435546875
Var (nonzero included): 22460.408203125
avg eig: 2.34825e+01 +/- 1.13662e+02
\entropy: 0.3736480176448822

test61 (nnz: 15):
Trace: 80.02381134033203
Var: 100.29574584960938
Var (nonzero included): 167.88282775878906
avg eig: 2.96384e+00 +/- 1.00148e+01
\entropy: 0.397367924451828

test76 (nnz: 15):
Trace: 78.53713989257812
Var: 82.30529022216797
Var (nonzero included): 135.96566772460938
avg eig: 2.90878e+00 +/- 9.07223e+00
\entropy: 0.48138687014579773

test178 (nnz: 15):
Trace: 461.69775390625
Var: 5258.583984375
Var (nonzero included): 9044.3857421875
avg eig: 1.70999e+01 +/- 7.25161e+01
\entropy: 0.3573622405529022

test107 (nnz: 15):
Trace: 1155.5782470703125
Var: 40139.56640625
Var (nonzero included): 69613.4609375
avg eig: 4.27992e+01 +/- 2.00349e+02
\entropy: 0.490171879529953

test87 (nnz: 15):
Trace: 1133.18212890625
Var: 41362.20703125
Var (nonzero included): 71915.484375
avg eig: 4.19697e+01 +/- 2.03377e+02
\en

(A1, b1):
Variance of gradients = 1.146881e+00; Varince of mean gradient 1.407727e-01 IQR of Variance (25,75) = 1.356635e-01

(A2, b2):
Variance of gradients = 1.017936e+00; Varince of mean gradient 1.154183e-02 IQR of Variance (25,75) = 1.315784e-01

(A3, b3):
Variance of gradients = 4.490731e-01; Varince of mean gradient 1.370976e-01 IQR of Variance (25,75) = 5.426976e-02

(A4, b4):
Variance of gradients = 1.361147e+00; Varince of mean gradient 7.919869e-02 IQR of Variance (25,75) = 1.532591e-01

(A5, b5):
Variance of gradients = 8.006192e-01; Varince of mean gradient 1.459432e-01 IQR of Variance (25,75) = 9.445225e-02

(A6, b6):
Variance of gradients = 6.027199e-01; Varince of mean gradient 6.308187e-02 IQR of Variance (25,75) = 8.518836e-02

(A7, b7):
Variance of gradients = 4.431704e-01; Varince of mean gradient 7.984268e-03 IQR of Variance (25,75) = 4.229897e-02

(A8, b8):
Variance of gradients = 6.303591e-01; Varince of mean gradient 8.015118e-02 IQR of Variance (25,75) = 6.523935e-02

(A9, b9):
Variance of gradients = 9.287970e-01; Varince of mean gradient 8.007182e-02 IQR of Variance (25,75) = 1.147567e-01

(A10, b10):
Variance of gradients = 6.220488e-01; Varince of mean gradient 3.316190e-02 IQR of Variance (25,75) = 1.458659e-01

Selected Dataset: A1, b1 with Variance Sum: 1.1468813545270882
second best:  A2 
17.96168054885399 0.005567407778323035 2.7413452576283843
Initial Learning Rate: 0.005567407778323035
Number of trainable parameters:  24